In [2]:
import socket
import re

In [2]:
print(socket.gethostbyname("www.google.com"))

216.58.196.100


In [1]:
import urlunshort3

In [2]:
url = "www.google.com"
print(urlunshort3.is_shortened(url))

AttributeError: module 'urlunshort3' has no attribute 'is_shortened'

In [3]:
url="http://www.legitimate.com//http://www.phishing.com"
print(re.findall("//", url))


['//', '//', '//']


In [5]:
import urllib

In [8]:
req = urllib.Request(url=url, headers=headers)
resp = urllib.urlopen(req, timeout=3)
redirected = resp.geturl() != url
print(redirected)

AttributeError: module 'urllib' has no attribute 'Request'